In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression,Lasso,LassoCV, Ridge, RidgeCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm 
from sklearn.metrics import r2_score
import pickle

: 

In [ ]:
df =pd.read_csv('Admission_Predict.csv')
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=['Serial No.'], inplace =True)

In [ ]:
df.describe()

## Splitting Features into Dependent & Independent Features, Data into Train, Test

In [ ]:
x = df.iloc[:,:-1]
y = df.iloc[ :,-1]

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
x_train,x_test, y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=45)

## Standardizing the Dependent Features data using Z-Transform


In [ ]:
scalar = StandardScaler()
x_train=scalar.fit_transform(x_train)
x_test= scalar.transform(x_test)

In [ ]:
pickle.dump(scalar,open("scaling_model.pkl","wb"))

In [ ]:
x_train

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_df = pd.DataFrame()
vif_df['VIF'] = [variance_inflation_factor(x_train,i) for i in range(x_train.shape[1])]
vif_df['Feature'] = x.columns
vif_df

In [ ]:
linear = LinearRegression()
linear.fit(x_train, y_train)

In [ ]:
predict_1= linear.predict(x_test)
predict_1 = predict_1.reshape(-1,1)
predict_1


In [ ]:
linear.score(x_test, y_test)

In [ ]:
## Adjusted R2_Score

def adjr2_score(x,y):
    sc=linear.score(x,y)
    n=x.shape[0]
    p=x.shape[1]
    adj = 1-(1-sc)*(n-1)/(n-p-1)
    return adj

In [ ]:
adjr2_score(x_test,y_test)

## Regularization

In [ ]:
lassocv = LassoCV(alphas=None, cv=50,max_iter=20000000)
lassocv.fit(x_train,y_train)

In [ ]:
lassocv.alpha_

In [ ]:
lasso_lr = Lasso(alpha=lassocv.alpha_)
lasso_lr.fit(x_train,y_train)
lasso_lr.score(x_test,y_test)

In [ ]:
ridgecv=RidgeCV(alphas=np.random.uniform(0,10,50), cv=15)
ridgecv.fit(x_train, y_train)

In [ ]:
ridgecv.alpha_

In [ ]:
ridge_lr =Ridge(alpha=ridgecv.alpha_)
ridge_lr.fit(x_train,y_train)
ridge_lr.score(x_test,y_test)

In [ ]:
elastic=ElasticNetCV(alphas=None, cv=10)
elastic.fit(x_train,y_train)

In [ ]:
elastic.alpha_

In [ ]:
elastic.l1_ratio_

In [ ]:
elastic_lr =ElasticNet(alpha=elastic.alpha_, l1_ratio=elastic.l1_ratio_)
elastic_lr.fit(x_train,y_train)
elastic_lr.score(x_test,y_test)

In [ ]:
pickle.dump(ridge_lr, open("ridge_regression_model.pkl","wb"))